In [41]:
using Dojo
using DojoEnvironments
using LinearAlgebra


In [42]:
quadrotor_env = get_environment(:quadrotor_waypoint; horizon=500)


DojoEnvironments.QuadrotorWaypoint{Float64, 500}

In [43]:
function controller!(environment, k)
    global current_waypoint_index
    check_waypoints!(environment, k)
end

controller! (generic function with 1 method)

In [44]:
function check_waypoints!(environment, k)

    global current_waypoint_index
    global des_pos
    waypoints = [[1.0, 1.0, 0.5], [2.0, 0.0, 0.5], [1.0, -1.0, 0.5], [0.0, 1.0, 0.5]]
    # waypoints = 
    # [[1,1;0.3],
    #     [2;0;0.3],
    #     [1;-1;0.3],
    #     [0;0;0.3],
    # ]
    current_waypoint_index = 1
 
    current_pos = get_state(environment)[1:3]
    des_pos = waypoints[current_waypoint_index]
    
 
    if norm(current_pos - des_pos) < 1e-1 
        # current_waypoint_index = current_waypoint_index % length(waypoints) + 1 
        current_waypoint_index = 2

        println("Next waypoint: ", waypoints[current_waypoint_index])
        des_pos = waypoints[current_waypoint_index]
    end
    println("Des_pos: ", des_pos)
    println("current_pos: ", current_pos)
    println("current_waypoint_index: ", current_waypoint_index)

    attitude_controller!(environment, k)

end

check_waypoints! (generic function with 1 method)

In [45]:
function state_provider!(environment)
    global des_pos
    global current_waypoint_index

    state = get_state(environment)
    position = state[1:3] # x, y, z
    orientation = state[4:6] # axis*angle
    linear_velocity = state[7:9] # vx, vy, vz
    angular_velocity = state[10:12] # ωx, ωy, ωz 

    current_roll = orientation[1]
    current_pitch = orientation[2]
    current_yaw = orientation[3]

    altitude = position[3]

    return position, orientation, linear_velocity, angular_velocity, current_roll, current_pitch, current_yaw, altitude, des_pos, current_waypoint_index
end

state_provider! (generic function with 1 method)

In [46]:
function position_controller!(environment, k)
    # here we will get the current_roll, current_pitch, current_yaw and thrust commands

    global des_pos
    global current_waypoint_index

    position, orientation, linear_velocity, angular_velocity, current_roll, current_pitch, current_yaw, altitude, des_pos, current_waypoint_index = state_provider!(environment)

    
    #This loop translates position errors into desired roll angles.
    K_P_1 = 0.04
    K_D_1= 0.1
    println("k ", k)
    println("Des_pos: ", des_pos)
    println("current_pos: ", position)
    
    des_roll = -(K_P_1 * (des_pos[2] - position[2]) + K_D_1 * (0 - linear_velocity[2]))
    des_pitch = K_P_1 * (des_pos[1] - position[1]) + K_D_1 * (0 - linear_velocity[1])

    return des_roll, des_pitch

end

position_controller! (generic function with 1 method)

In [47]:
function attitude_controller!(environment, k)
    # here we will calculate the controll commands using ref and current state 
    global des_pos
    global current_waypoint_index

    position, orientation, linear_velocity, angular_velocity, current_roll, current_pitch, current_yaw, altitude, des_pos, current_waypoint_index = state_provider!(environment)

    K_P_2 = 4.0
    K_D_2 = 1
    error_z = des_pos[3] - altitude
    v_z = linear_velocity[3]
    des_yaw = 0.0
    thrust_feedforward = 20 * 5.1 * 1/sqrt(4)

    des_roll, des_pitch = position_controller!(environment, k)
    println("des_roll: ", des_roll, " des_pitch: ", des_pitch, " des_yaw: ", des_yaw)

    output_roll = K_P_2 * (des_roll - current_roll) + K_D_2 * (0 - angular_velocity[1])
    output_pitch = K_P_2 * (des_pitch - current_pitch) + K_D_2 * (0 - angular_velocity[2])  
    output_yaw = K_P_2 * (des_yaw - current_yaw) + K_D_2 * (0 - angular_velocity[3])
    output_thrust = (10*error_z - 10* v_z )+ thrust_feedforward
    # K_p_thrust * (altitude_ref - altitude) + K_d_thrust * (0 - get_state(environment)[9])+ thrust_feedforward
    println("output_roll: ", output_roll, " output_pitch: ", output_pitch, " output_yaw: ", output_yaw, " output_thrust: ", output_thrust)

    u = MMA!(output_roll, output_pitch, output_yaw, output_thrust)
    print("u: ", u)
    set_input!(environment, u)  
end

attitude_controller! (generic function with 1 method)

In [48]:

function MMA!(output_roll, output_pitch, output_yaw, output_thrust)
    u = zeros(4)
    u[1] = output_thrust + output_yaw + output_pitch + output_roll
    u[2] = output_thrust - output_yaw + output_pitch - output_roll
    u[3] = output_thrust - output_yaw - output_pitch + output_roll
    u[4] = output_thrust + output_yaw - output_pitch - output_roll
    return u
end

MMA! (generic function with 1 method)

In [49]:
initialize!(quadrotor_env, :quadrotor)
simulate!(quadrotor_env, controller!; record=true)


Storage for 500 steps of 5 bodies.


In [50]:
### Visualize
vis = visualize(quadrotor_env)
render(vis)


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("Background" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}("top_color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x42, 0x61, 0x63, 0x6b, 0x67, 0x72, 0x6f, 0x75, 0x6e, 0x64], "bottom_color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x42, 0x61, 0x63, 0x6b, 0x67, 0x72, 0x6f, 0x75, 0x6e, 0x64]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Grid" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}("visible" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x70, 0x61, 0x74, 0x68, 0xa5, 0x2f, 0x47, 0x72, 0x69, 0x64]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("waypoints" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("waypoint1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x31], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x33], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "floor" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("bodies" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("rotor_3__id_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x5f, 0x33, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x31], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "rotor_1__id_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x5f, 0x31, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "base_link__id_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6c, 0x69, 0x6e, 0x6b, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "rotor_0__id_5" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 